In [1]:
import geopandas as gpd

In [8]:
zone_1_path = 'C:/WSP_projects/Freight/zone_shapefiles/GBFM/Zones.shp'
zone_2_path = 'C:/WSP_projects/Freight/zone_shapefiles/NoHAM/noham_zones_freeze_2.10.shp'
outpath = 'C:/WSP_projects/Freight/local_freight_tool/Outputs/Zone correspondence test'
zone_1_name = 'gbfm'
zone_2_name = 'noham'

In [9]:
gbfm_lookup = {'ID': 'zone_id'}
noham_lookup = {
    'zone_id': 'empty',
    'unique_id': 'zone_id'
    }

In [10]:
zone_1 = gpd.read_file(zone_1_path)
zone_2 = gpd.read_file(zone_2_path)

In [11]:
zone_list = [zone_1, zone_2]
zone_names = [zone_1_name, zone_2_name]

In [15]:
for i in range(2):
        
    if 'ID' in zone_list[i].columns:
        # this uses GBFM zone ID column name
        zone_list[i] = zone_list[i].rename(columns=gbfm_lookup)
    elif 'unique_id' in zone_list[i].columns:
        # this uses NoHAM zone ID column name
        zone_list[i] = zone_list[i].rename(columns=noham_lookup)
    else:
        print('no lookup for this zone system, need to know column names')

    zone_list[i] = zone_list[i].rename(
        columns = {'zone_id': f'{zone_names[i]}_zone_id'}
    )
    zone_list[i][f'{zone_names[i]}_area'] = zone_list[i].area

    # set gbfm crs data to same as noham crs data (they should be the
    # same but gbfm data is incomplete)
    # TODO might change this to just set all to this crs by default
    if not zone_list[i].crs:
        zone_list[i].crs = 'EPSG:27700'

no lookup for this zone system, need to know column names
no lookup for this zone system, need to know column names


In [18]:
zone_overlay = gpd.overlay(zone_list[0], zone_list[1], how='intersection').reset_index()
zone_overlay['intersection_area'] = zone_overlay.area

In [19]:
spatial_zone_correspondence = zone_overlay[[f'{zone_names[0]}_zone_id',
                            f'{zone_names[1]}_zone_id']]
spatial_zone_correspondence[f'{zone_names[0]}_to_{zone_names[1]}'] = (zone_overlay['intersection_area'] /
                                                                    zone_overlay[f'{zone_names[0]}_area'])
spatial_zone_correspondence[f'{zone_names[1]}_to_{zone_names[0]}'] = (zone_overlay['intersection_area'] /
                                                                    zone_overlay[f'{zone_names[1]}_area'])

<ipython-input-19-7a826ce4aa35>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spatial_zone_correspondence[f'{zone_names[0]}_to_{zone_names[1]}'] = (zone_overlay['intersection_area'] /
<ipython-input-19-7a826ce4aa35>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spatial_zone_correspondence[f'{zone_names[1]}_to_{zone_names[0]}'] = (zone_overlay['intersection_area'] /


In [ ]:
spatial_zone